# 네이버 데이터랩 분야1 전체 자동 스크래핑 프로그램 (구글 드라이브 저장 버전)

## 프로그램 설명
네이버 데이터랩의 분야1 카테고리 전체 검색어를 자동으로 수집하고 구글 드라이브에 업로드하는 프로그램입니다.

## 주요 기능
- 네이버 데이터랩 분야1 카테고리 전체 검색어 자동 수집
- 구글 드라이브 자동 업로드
- 엑셀 파일 형식으로 저장
- 자동 페이지네이션 처리

## 사용 방법
1. 프로그램 실행
2. 자동으로 분야1 카테고리 검색어 수집
3. 구글 드라이브에 자동 업로드

## 필요 라이브러리
- selenium: 웹 크롤링
- openpyxl: 엑셀 파일 처리
- google-api-python-client: 구글 드라이브 API

## 주의사항
- Google Drive API 인증이 필요합니다 (client_secret.json 파일 필요)
- 네이버 데이터랩 페이지 구조 변경 시 수정이 필요할 수 있습니다

In [24]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from openpyxl import Workbook
import time
import os
from datetime import datetime
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseUpload
import io
import pickle

# Google Drive API 관련 상수
SCOPES = ['https://www.googleapis.com/auth/drive.file']
GOOGLE_DRIVE_FOLDER_ID = "1-8kck8fizzDI5Zc9SmlHXxu-d9bUEkcY"  # 고정된 폴더 ID

def get_google_drive_service():
    """Google Drive API 인증 및 서비스 객체 생성"""
    creds = None
    
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)
    
    return build('drive', 'v3', credentials=creds)

def setup_excel_file():
    current_date = datetime.now().strftime("%Y%m%d")
    base_file_name = input("저장할 Excel 파일의 기본 이름을 입력하세요 (확장자 제외): ")
    excel_file_name = f"{base_file_name}_{current_date}.xlsx"
    return excel_file_name

def upload_to_drive(service, file_content, file_name):
    """Google Drive에 파일 업로드"""
    try:
        file_metadata = {
            'name': file_name,
            'parents': [GOOGLE_DRIVE_FOLDER_ID]
        }
        
        fh = io.BytesIO(file_content)
        media = MediaIoBaseUpload(fh,
                                mimetype='application/vnd.openxmlformats-officedocument.spreadsheetml.sheet',
                                resumable=True)
        
        file = service.files().create(body=file_metadata,
                                    media_body=media,
                                    fields='id').execute()
        
        print(f"파일이 성공적으로 Google Drive에 업로드되었습니다. File ID: {file.get('id')}")
        return file.get('id')
    
    except Exception as e:
        print(f"Google Drive 업로드 중 오류 발생: {e}")
        return None

def setup_webdriver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    driver.get("https://datalab.naver.com/shoppingInsight/sCategory.naver")
    return driver

def wait_for_element(driver, by, value, timeout=20):
    return WebDriverWait(driver, timeout).until(EC.element_to_be_clickable((by, value)))

def get_category_names(driver, dropdown_path, options_xpath):
    dropdown = wait_for_element(driver, By.XPATH, dropdown_path)
    dropdown.click()
    time.sleep(1)
    options = driver.find_elements(By.XPATH, options_xpath)
    category_names = [option.text.strip() for option in options]
    dropdown.click()
    return category_names

def select_category(driver, dropdown_path, option_xpath):
    dropdown = wait_for_element(driver, By.XPATH, dropdown_path)
    dropdown.click()
    option = wait_for_element(driver, By.XPATH, option_xpath)
    option.click()
    time.sleep(1)

def select_all_options(driver):
    options = [
        '//*[@id="18_device_0"]',
        '//*[@id="19_gender_0"]',
        '//*[@id="20_age_0"]'
    ]
    for option in options:
        element = driver.find_element(By.XPATH, option)
        driver.execute_script("arguments[0].scrollIntoView(true);", element)
        element.click()
    time.sleep(2)

def scrape_data(driver, category1, period, ws):
    for i in range(25):
        time.sleep(1)
        for j in range(1, 21):
            try:
                path = f'//*[@id="content"]/div[2]/div/div[2]/div[2]/div/div/div[1]/ul/li[{j}]/a'
                result = driver.find_element(By.XPATH, path).text
                rank, keyword = result.split('\n')
                row_data = [int(rank), keyword, period, category1]
                print(row_data)
                ws.append(row_data)
            except Exception as e:
                print(f"Error occurred while scraping page {i+1}, item {j}: {e}")
                break

        try:
            next_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, 'a.btn_page_next'))
            )
            next_button.click()
        except Exception as e:
            print(f"Failed to move to next page after page {i+1}: {e}")
            break

def main():
    try:
        service = get_google_drive_service()
    except Exception as e:
        print(f"Google Drive 서비스 초기화 실패: {e}")
        return

    excel_file_name = setup_excel_file()
    driver = setup_webdriver()
    wait = WebDriverWait(driver, 20)

    input("엔터 키를 누르면 스크래핑을 시작합니다...")

    wb = Workbook()
    ws = wb.active
    ws.title = "쇼핑 인사이트 데이터"
    ws.append(['순위', '인기검색어', '기간', '분야1'])

    dropdown1_path = '//*[@id="content"]/div[2]/div/div[1]/div/div/div[1]/div/div[1]/span'
    category1_names = get_category_names(driver, dropdown1_path, '//*[@id="content"]/div[2]/div/div[1]/div/div/div[1]/div/div[1]/ul/li/a')

    for option1_num, category1_name in enumerate(category1_names, start=1):
        try:
            select_category(driver, dropdown1_path, f'//*[@id="content"]/div[2]/div/div[1]/div/div/div[1]/div/div[1]/ul/li[{option1_num}]/a')
            selected_category1 = driver.find_element(By.XPATH, dropdown1_path).text.strip()
            
            if selected_category1 != category1_name:
                print(f"Selected category1 '{selected_category1}' does not match expected category '{category1_name}'")
                continue

            select_all_options(driver)

            search_button = wait_for_element(driver, By.XPATH, '//*[@id="content"]/div[2]/div/div[1]/div/a')
            search_button.click()
            
            time.sleep(3)  # 조회후 대기시간 설정

            period = driver.find_element(By.CSS_SELECTOR, 'span.title_sub').text
            scrape_data(driver, selected_category1, period, ws)


        except Exception as e:
            print(f"Error occurred while processing category1 option {option1_num}: {e}")

    try:
        excel_buffer = io.BytesIO()
        wb.save(excel_buffer)
        excel_content = excel_buffer.getvalue()
        
        file_id = upload_to_drive(service, excel_content, excel_file_name)
        
        if file_id:
            print(f"파일이 Google Drive에 성공적으로 업로드되었습니다.")
            print(f"파일 링크: https://drive.google.com/file/{file_id}/view")
        else:
            print("파일 업로드 실패")
            
    except Exception as e:
        print(f"파일 저장 중 오류 발생: {e}")

    driver.quit()

if __name__ == "__main__":
    main()

저장할 Excel 파일의 기본 이름을 입력하세요 (확장자 제외): 2024년10월_분야1_인기검색어
엔터 키를 누르면 스크래핑을 시작합니다...
[1, '원피스', '2024.10.01. ~ 2024.10.30.', '패션의류']
[2, '트위드자켓', '2024.10.01. ~ 2024.10.30.', '패션의류']
[3, '스웨이드자켓', '2024.10.01. ~ 2024.10.30.', '패션의류']
[4, '맨투맨', '2024.10.01. ~ 2024.10.30.', '패션의류']
[5, '가디건', '2024.10.01. ~ 2024.10.30.', '패션의류']
[6, '후드집업', '2024.10.01. ~ 2024.10.30.', '패션의류']
[7, '패딩조끼', '2024.10.01. ~ 2024.10.30.', '패션의류']
[8, '경량패딩', '2024.10.01. ~ 2024.10.30.', '패션의류']
[9, '후드티', '2024.10.01. ~ 2024.10.30.', '패션의류']
[10, '잠옷', '2024.10.01. ~ 2024.10.30.', '패션의류']
[11, '여성가디건', '2024.10.01. ~ 2024.10.30.', '패션의류']
[12, '가죽자켓', '2024.10.01. ~ 2024.10.30.', '패션의류']
[13, '남자맨투맨', '2024.10.01. ~ 2024.10.30.', '패션의류']
[14, '트렌치코트', '2024.10.01. ~ 2024.10.30.', '패션의류']
[15, '니트원피스', '2024.10.01. ~ 2024.10.30.', '패션의류']
[16, '바람막이', '2024.10.01. ~ 2024.10.30.', '패션의류']
[17, '써스데이아일랜드원피스', '2024.10.01. ~ 2024.10.30.', '패션의류']
[18, '니트', '2024.10.01. ~ 2024.10.30.', '패션의류']
[19, '나이키바람막이', '2024.

[161, '내셔널지오그래픽', '2024.10.01. ~ 2024.10.30.', '패션의류']
[162, '쉬즈미스자켓', '2024.10.01. ~ 2024.10.30.', '패션의류']
[163, '다이나핏', '2024.10.01. ~ 2024.10.30.', '패션의류']
[164, '써스데이아일랜드가디건', '2024.10.01. ~ 2024.10.30.', '패션의류']
[165, '모조에스핀자켓', '2024.10.01. ~ 2024.10.30.', '패션의류']
[166, '울자켓', '2024.10.01. ~ 2024.10.30.', '패션의류']
[167, '케이프코트', '2024.10.01. ~ 2024.10.30.', '패션의류']
[168, '베네통니트', '2024.10.01. ~ 2024.10.30.', '패션의류']
[169, '캘빈클라인', '2024.10.01. ~ 2024.10.30.', '패션의류']
[170, '남자후드티', '2024.10.01. ~ 2024.10.30.', '패션의류']
[171, 'cp컴퍼니', '2024.10.01. ~ 2024.10.30.', '패션의류']
[172, '커버낫맨투맨', '2024.10.01. ~ 2024.10.30.', '패션의류']
[173, '쥬시쥬디원피스', '2024.10.01. ~ 2024.10.30.', '패션의류']
[174, '청자켓', '2024.10.01. ~ 2024.10.30.', '패션의류']
[175, '브룩스브라더스', '2024.10.01. ~ 2024.10.30.', '패션의류']
[176, '시슬리원피스', '2024.10.01. ~ 2024.10.30.', '패션의류']
[177, '오르시떼여성잠옷', '2024.10.01. ~ 2024.10.30.', '패션의류']
[178, '마인드브릿지', '2024.10.01. ~ 2024.10.30.', '패션의류']
[179, '폴로셔츠', '2024.10.01. ~ 2024.10.30.', '패션의

[321, '심리스팬티', '2024.10.01. ~ 2024.10.30.', '패션의류']
[322, '후드니트', '2024.10.01. ~ 2024.10.30.', '패션의류']
[323, '폴로후드집업', '2024.10.01. ~ 2024.10.30.', '패션의류']
[324, '브라', '2024.10.01. ~ 2024.10.30.', '패션의류']
[325, '오즈세컨', '2024.10.01. ~ 2024.10.30.', '패션의류']
[326, '버버리트렌치코트', '2024.10.01. ~ 2024.10.30.', '패션의류']
[327, '트위드셋업', '2024.10.01. ~ 2024.10.30.', '패션의류']
[328, '여성셔츠', '2024.10.01. ~ 2024.10.30.', '패션의류']
[329, '하프코트', '2024.10.01. ~ 2024.10.30.', '패션의류']
[330, '겨울원피스', '2024.10.01. ~ 2024.10.30.', '패션의류']
[331, '모조에스핀코트', '2024.10.01. ~ 2024.10.30.', '패션의류']
[332, '톰브라운니트', '2024.10.01. ~ 2024.10.30.', '패션의류']
[333, '뉴발란스바람막이', '2024.10.01. ~ 2024.10.30.', '패션의류']
[334, '코듀로이반바지', '2024.10.01. ~ 2024.10.30.', '패션의류']
[335, '단톤', '2024.10.01. ~ 2024.10.30.', '패션의류']
[336, '퀼팅자켓', '2024.10.01. ~ 2024.10.30.', '패션의류']
[337, 'jj지고트', '2024.10.01. ~ 2024.10.30.', '패션의류']
[338, '유라타임', '2024.10.01. ~ 2024.10.30.', '패션의류']
[339, '여자맨투맨', '2024.10.01. ~ 2024.10.30.', '패션의류']
[340, '홈웨어',

[481, '럭키슈에뜨자켓', '2024.10.01. ~ 2024.10.30.', '패션의류']
[482, '엄브로패딩', '2024.10.01. ~ 2024.10.30.', '패션의류']
[483, '타미힐피거니트', '2024.10.01. ~ 2024.10.30.', '패션의류']
[484, '시슬리코트', '2024.10.01. ~ 2024.10.30.', '패션의류']
[485, '후아유가디건', '2024.10.01. ~ 2024.10.30.', '패션의류']
[486, '헤지스가디건', '2024.10.01. ~ 2024.10.30.', '패션의류']
[487, '드레스대여', '2024.10.01. ~ 2024.10.30.', '패션의류']
[488, '쥬크원피스', '2024.10.01. ~ 2024.10.30.', '패션의류']
[489, '나이키후리스', '2024.10.01. ~ 2024.10.30.', '패션의류']
[490, 'cp컴퍼니패딩', '2024.10.01. ~ 2024.10.30.', '패션의류']
[491, '남자코트', '2024.10.01. ~ 2024.10.30.', '패션의류']
[492, '엄브로아노락', '2024.10.01. ~ 2024.10.30.', '패션의류']
[493, '나이스클랍니트', '2024.10.01. ~ 2024.10.30.', '패션의류']
[494, 'bcbg원피스', '2024.10.01. ~ 2024.10.30.', '패션의류']
[495, '에디바우어기모조거팬츠', '2024.10.01. ~ 2024.10.30.', '패션의류']
[496, '로엠자켓', '2024.10.01. ~ 2024.10.30.', '패션의류']
[497, '타미가디건', '2024.10.01. ~ 2024.10.30.', '패션의류']
[498, '레이어드원피스', '2024.10.01. ~ 2024.10.30.', '패션의류']
[499, '라인어디션', '2024.10.01. ~ 2024.10.30.', 

[141, '아디다스슬리퍼', '2024.10.01. ~ 2024.10.30.', '패션잡화']
[142, '진주목걸이', '2024.10.01. ~ 2024.10.30.', '패션잡화']
[143, '셀린느', '2024.10.01. ~ 2024.10.30.', '패션잡화']
[144, '잔스포츠미니백팩', '2024.10.01. ~ 2024.10.30.', '패션잡화']
[145, '닥터마틴', '2024.10.01. ~ 2024.10.30.', '패션잡화']
[146, '명품가방', '2024.10.01. ~ 2024.10.30.', '패션잡화']
[147, '호보백', '2024.10.01. ~ 2024.10.30.', '패션잡화']
[148, '스와로브스키목걸이', '2024.10.01. ~ 2024.10.30.', '패션잡화']
[149, '몽블랑벨트', '2024.10.01. ~ 2024.10.30.', '패션잡화']
[150, '버버리스카프', '2024.10.01. ~ 2024.10.30.', '패션잡화']
[151, '메트로시티가방', '2024.10.01. ~ 2024.10.30.', '패션잡화']
[152, '실내화', '2024.10.01. ~ 2024.10.30.', '패션잡화']
[153, '넥타이', '2024.10.01. ~ 2024.10.30.', '패션잡화']
[154, '진주귀걸이', '2024.10.01. ~ 2024.10.30.', '패션잡화']
[155, '뉴발530', '2024.10.01. ~ 2024.10.30.', '패션잡화']
[156, '입생로랑가방', '2024.10.01. ~ 2024.10.30.', '패션잡화']
[157, '버킷백', '2024.10.01. ~ 2024.10.30.', '패션잡화']
[158, '샘소나이트캐리어', '2024.10.01. ~ 2024.10.30.', '패션잡화']
[159, '고야드가방', '2024.10.01. ~ 2024.10.30.', '패션잡화']
[160, '프

...

Error occurred while scraping page 16, item 1: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="content"]/div[2]/div/div[2]/div[2]/div/div/div[1]/ul/li[1]/a"}
  (Session info: chrome=129.0.6668.101); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF6DABAB095+29557]
	(No symbol) [0x00007FF6DAB1FA50]
	(No symbol) [0x00007FF6DA9DB56A]
	(No symbol) [0x00007FF6DAA2F695]
	(No symbol) [0x00007FF6DAA2F8EC]
	(No symbol) [0x00007FF6DAA7B777]
	(No symbol) [0x00007FF6DAA571CF]
	(No symbol) [0x00007FF6DAA7851C]
	(No symbol) [0x00007FF6DAA56F33]
	(No symbol) [0x00007FF6DAA2116F]
	(No symbol) [0x00007FF6DAA222D1]
	GetHandleVerifier [0x00007FF6DAEDC96D+3378253]
	GetHandleVerifier [0x00007FF6DAF28497+3688311]
	GetHandleVerifier [0x00007FF6DAF1D1CB+3642539]
	GetHandleVerifier [0x00007FF6DAC6A6B6+813462]
	(No symbol) [0x00007FF6

Error occurred while scraping page 23, item 1: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="content"]/div[2]/div/div[2]/div[2]/div/div/div[1]/ul/li[1]/a"}
  (Session info: chrome=129.0.6668.101); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF6DABAB095+29557]
	(No symbol) [0x00007FF6DAB1FA50]
	(No symbol) [0x00007FF6DA9DB56A]
	(No symbol) [0x00007FF6DAA2F695]
	(No symbol) [0x00007FF6DAA2F8EC]
	(No symbol) [0x00007FF6DAA7B777]
	(No symbol) [0x00007FF6DAA571CF]
	(No symbol) [0x00007FF6DAA7851C]
	(No symbol) [0x00007FF6DAA56F33]
	(No symbol) [0x00007FF6DAA2116F]
	(No symbol) [0x00007FF6DAA222D1]
	GetHandleVerifier [0x00007FF6DAEDC96D+3378253]
	GetHandleVerifier [0x00007FF6DAF28497+3688311]
	GetHandleVerifier [0x00007FF6DAF1D1CB+3642539]
	GetHandleVerifier [0x00007FF6DAC6A6B6+813462]
	(No symbol) [0x00007FF6

[101, '쇼펜하우어', '2024.10.01. ~ 2024.10.30.', '도서']
[102, '너에게들려주는단단한말', '2024.10.01. ~ 2024.10.30.', '도서']
[103, '소설추천', '2024.10.01. ~ 2024.10.30.', '도서']
[104, '혜윰모의고사', '2024.10.01. ~ 2024.10.30.', '도서']
[105, '기분이태도가되지말자', '2024.10.01. ~ 2024.10.30.', '도서']
[106, '디에센셜한강', '2024.10.01. ~ 2024.10.30.', '도서']
[107, '부자아빠가난한아빠', '2024.10.01. ~ 2024.10.30.', '도서']
[108, '회색인간', '2024.10.01. ~ 2024.10.30.', '도서']
[109, '압도하라타이거즈', '2024.10.01. ~ 2024.10.30.', '도서']
[110, '채식주의자줄거리', '2024.10.01. ~ 2024.10.30.', '도서']
[111, '파친코책', '2024.10.01. ~ 2024.10.30.', '도서']
[112, '정유정', '2024.10.01. ~ 2024.10.30.', '도서']
[113, '내여자의열매', '2024.10.01. ~ 2024.10.30.', '도서']
[114, '우리가빛의속도로갈수없다면', '2024.10.01. ~ 2024.10.30.', '도서']
[115, '행복의기원', '2024.10.01. ~ 2024.10.30.', '도서']
[116, '돈의속성', '2024.10.01. ~ 2024.10.30.', '도서']
[117, '말투만바꿨을뿐인데', '2024.10.01. ~ 2024.10.30.', '도서']
[118, 'humanacts', '2024.10.01. ~ 2024.10.30.', '도서']
[119, '모순양귀자', '2024.10.01. ~ 2024.10.30.', '도서']
[120, 'basicgram

[241, '소방안전관리자2급', '2024.10.01. ~ 2024.10.30.', '도서']
[242, '이반일리치의죽음', '2024.10.01. ~ 2024.10.30.', '도서']
[243, '인간관계론', '2024.10.01. ~ 2024.10.30.', '도서']
[244, '수학의정석', '2024.10.01. ~ 2024.10.30.', '도서']
[245, '내몸혁명', '2024.10.01. ~ 2024.10.30.', '도서']
[246, '맡겨진소녀', '2024.10.01. ~ 2024.10.30.', '도서']
[247, '두사람의인터내셔널', '2024.10.01. ~ 2024.10.30.', '도서']
[248, '투자자산운용사', '2024.10.01. ~ 2024.10.30.', '도서']
[249, '다정한것이살아남는다', '2024.10.01. ~ 2024.10.30.', '도서']
[250, '정유정작가', '2024.10.01. ~ 2024.10.30.', '도서']
[251, '채식주의', '2024.10.01. ~ 2024.10.30.', '도서']
[252, '청소년책추천', '2024.10.01. ~ 2024.10.30.', '도서']
[253, '시그니처모의고사', '2024.10.01. ~ 2024.10.30.', '도서']
[254, '동물농장책', '2024.10.01. ~ 2024.10.30.', '도서']
[255, '패스코드투자자산운용사', '2024.10.01. ~ 2024.10.30.', '도서']
[256, '원페이지교육학', '2024.10.01. ~ 2024.10.30.', '도서']
[257, '이상한정상가족', '2024.10.01. ~ 2024.10.30.', '도서']
[258, '은하수잡화점', '2024.10.01. ~ 2024.10.30.', '도서']
[259, '노벨문학상수상작', '2024.10.01. ~ 2024.10.30.', '도서']
[260, '유아식책', '2

[401, '수학이생명의언어라면', '2024.10.01. ~ 2024.10.30.', '도서']
[402, '모순책', '2024.10.01. ~ 2024.10.30.', '도서']
[403, '영어독립voca3000', '2024.10.01. ~ 2024.10.30.', '도서']
[404, '일본어무작정따라하기', '2024.10.01. ~ 2024.10.30.', '도서']
[405, '한강몽고반점', '2024.10.01. ~ 2024.10.30.', '도서']
[406, '해독혁명', '2024.10.01. ~ 2024.10.30.', '도서']
[407, '잘했고잘하고있고잘될것이다', '2024.10.01. ~ 2024.10.30.', '도서']
[408, '줄리의그림자', '2024.10.01. ~ 2024.10.30.', '도서']
[409, '나는나로살기로했다', '2024.10.01. ~ 2024.10.30.', '도서']
[410, '일본어책', '2024.10.01. ~ 2024.10.30.', '도서']
[411, '총균쇠책', '2024.10.01. ~ 2024.10.30.', '도서']
[412, '모니카와케이티', '2024.10.01. ~ 2024.10.30.', '도서']
[413, '정몽규', '2024.10.01. ~ 2024.10.30.', '도서']
[414, '유튜브', '2024.10.01. ~ 2024.10.30.', '도서']
[415, '우리아이꼭지켜줄게', '2024.10.01. ~ 2024.10.30.', '도서']
[416, '죽고싶지만떡볶이는먹고싶어', '2024.10.01. ~ 2024.10.30.', '도서']
[417, '공통수학1', '2024.10.01. ~ 2024.10.30.', '도서']
[418, '몽글몽글편의점', '2024.10.01. ~ 2024.10.30.', '도서']
[419, '나태주', '2024.10.01. ~ 2024.10.30.', '도서']
[420, '탕비실', '